In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
import io

# Load the trained model
model = tf.keras.models.load_model('lstm_autoencoder_model.keras')

# Initialize FastAPI app
app = FastAPI()



In [5]:
# Define request and response models
class PredictionRequest(BaseModel):
    csv_data: str

class PredictionResponse(BaseModel):
    alert: bool
    date_range: str

# Helper function to read CSV from string
def read_csv_from_string(csv_string: str):
    return pd.read_csv(io.StringIO(csv_string), parse_dates=['Date'], index_col='Date')

@app.post("/predict", response_model=PredictionResponse)
async def predict_anomaly(file: UploadFile = File(...)):
    content = await file.read()
    df = pd.read_csv(io.BytesIO(content), parse_dates=['Date'], index_col='Date')
    # seq_len: int = Query(10, description="Length of the sequence") 
    # threshold: float = Query(0.1, description="Threshold for anomaly detection (might change in case of further data drift)")

    # Preprocess the data to match the input format of the model
    sequences = create_sequences(df.values, seq_length=3)

    # Make predictions
    predictions = model.predict(sequences)

    # Calculate loss
    data_loss = tf.keras.losses.mae(predictions, sequences)
    data_loss_mean = tf.reduce_mean(data_loss, axis=1)

    # Check for anomalies (you need to define your threshold)
    threshold = 2  # Example threshold
    anomalies = data_loss_mean > threshold

    # Extract date range for anomalies
    anomalous_dates = df.index[seq_length:][anomalies]
    date_range = f"{anomalous_dates.min()} - {anomalous_dates.max()}"

    return PredictionResponse(alert=any(anomalies), date_range=date_range)

# Helper function to create sequences
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length + 1):
        sequences.append(data[i:i + seq_length])
    return np.array(sequences)

# Run the app with: `uvicorn api:app --reload`


In [7]:
uvicorn api_gen:app --reload


SyntaxError: invalid syntax (1633062132.py, line 1)

In [ ]:
curl -X POST "http://127.0.0.1:8000/predict" -F "file=@anomaly_detection.csv"
# In Postman, you can create a new POST request with the following details:
# URL: http://127.0.0.1:8000/predict
# Method: POST
# Under the "Body" tab, select "form-data"
# Add a key named "file" and set the type to "File"
# Choose the file "anomaly_detection.csv" from your file system



In [9]:
import requests

url = "http://127.0.0.1:8000/predict"
params = {'seq_len': 12, 'threshold': 0.15}
files = {'file': open('anomaly_detection2.csv', 'rb')}

response = requests.post(url, params=params, files=files)
print(response.json())


JSONDecodeError: Expecting value: line 1 column 1 (char 0)